# MD-WERP Strategic Project 11.2


Requirements:

Need a Google Earth Engine login, subject to their Terms and Conditions

In [ ]:
# !pip install earthengine-api --upgrade

## Import modules

In [ ]:
import ee
import folium
import time

## Connect to Google Earth Engine

In [ ]:
ee.Authenticate()
ee.Initialize()

## Define user variables

Import datasets

In [ ]:
bounds = ee.FeatureCollection('users/Projects/Hunt/Darling_ANAEv3_LM_1000m_diss').geometry()

Set user defined variables

In [ ]:
# Test mode
test_mode = 'no' # yes to not submit tasks, no to submit tasks

# Set date range 
start_date = 2022
end_date = 2023

# Set parameters for Image Collections
index = 'mndwi'
threshold = 0 # water index threshold
cloud_percent = 20
collection = 'landsat'
slc_data = 'no' # yes for include data, no for through affected data away
clip_edges = 'yes' # yes to clip by 6km, no to not

# This code should be monthly only, but you can do year and day
timeframe = ee.String('month') 
advance_overlap = 1 

# Set export variables
code_output = 'water_persistence' # monthly_water or water_persistence
image = 'percent' # Choose from 'percent', 'clear_count' or 'water_count'
bucket = 'sharebucketgee'
maxPixels = 1e13
crs = 'EPSG:32654'
crsTransform = [30, 0, 564285, 0, -30, -3395385]
fileNamePrefix = 'MDWERP112/January_2023/'#'MDWERP11/Project_112/Persistence_summaries/' #'Project_112/'

# Import collections
l9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
l7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
l5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")

## Define functions

In [ ]:
# Define a function to scale the data and mask unwanted pixels.
def maskL457sr(image):
    '''
    maskL457sr from https://developers.google.com/earth-engine/guides/ic_visualization
    '''
    # Bit 0 - Fill
    # Bit 1 - Dilated Cloud
    # Bit 2 - Unused
    # Bit 3 - Cloud
    # Bit 4 - Cloud Shadow
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)

    # Apply the scaling factors to the appropriate bands.
    # opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    # thermalBand = image.select('ST_B6').multiply(0.00341802).add(149.0)
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)

    # Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True)\
        .addBands(thermalBands, None, True)\
        .updateMask(qaMask)\
        .updateMask(saturationMask)

# Landsat 8 and 9
def renameBands89(img):
    return img.select(['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7']).rename(['blue','green','red','nir','swir1','swir2'])

# Landsat (4), 5 and 7
def renameBands57(img):
    return img.select(['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B7']).rename(['blue','green','red','nir','swir1','swir2'])

# Function to clip edges of Landsat
def clipEdges(img):
    return img.clip(img.geometry().buffer(-6000))

# Function to add MNDWI
def addMNDWI(image):
    b1 = image.select('green')
    b2 = image.select('swir1')
    new_band = b1.subtract(b2).divide(b1.add(b2)).rename('mndwi')
    return image.addBands(new_band)

# Function to keep only water
def onlyW(image):
    water = image.select(index).gt(threshold).rename('water')
    return image.addBands(water).selfMask()

# Function to keep only clear
def onlyClear(image):
    clear = image.select('red').gt(0).rename('clear')
    return image.addBands(clear).selfMask()

# Funciton to mask bad pixels
def maskBad(image):
    bad_mask = image.select(['blue','green','red','nir','swir1','swir2'])\
                    .reduce(ee.Reducer.min())\
                    .gt(0)
    return image.mask(bad_mask)

## Process the images

### Export monthly water images

In [ ]:
if code_output == 'monthly_water':
    ### Set up datelist to loop through
    years = list(range(start_date,end_date))

    for year in years:

        # User variables
        start = str(year)+'-01-01'
        end = str(year+1)+'-01-01'
        print(start,end)

        try:
            # Do collection specific mapping
            l5c = l5.map(maskL457sr).map(renameBands57)
            if slc_data == 'yes':
                l7c = l7.map(maskL457sr).map(renameBands57)
            elif slc_data == 'no':
                l7c = l7.map(maskL457sr).map(renameBands57).filterDate('1999-01-01','2003-05-31') # Remove SLC error affected images
            l8c = l8.map(maskL457sr).map(renameBands89)
            l9c = l9.map(maskL457sr).map(renameBands89)
            if clip_edges == 'yes':
                col = (l5c.map(clipEdges)).merge(l7c.map(clipEdges)).merge(l8c).merge(l9c)\
                         .filterDate(start,end)\
                         .filterBounds(bounds)\
                         .filter(ee.Filter.lt('CLOUD_COVER',cloud_percent))\
                         .map(addMNDWI)\
                         .map(maskBad)
            elif clip_edges == 'no':
                col = l5c.merge(l7c).merge(l8c).merge(l9c)\
                         .filterDate(start,end)\
                         .filterBounds(bounds)\
                         .filter(ee.Filter.lt('CLOUD_COVER',cloud_percent))\
                         .map(addMNDWI)\
                         .map(maskBad)
            

            #################################################################
            #/ Monthly water presence #/


            if code_output == 'monthly_water':
                # Run this

                #Filter collection and apply functions - water and count
                daily_mosaics = col.map(onlyW)\
                                       .map(onlyClear)

                # Now we need the make monthly summaries
                # Filter for each month, sum, and then sum over the basin
                start = ee.Date(start)
                end = ee.Date(end)
                months = end.difference(start, timeframe)
                datelist = ee.List.sequence(0, months.int()).map(lambda count: start.advance(count, timeframe))

                # This function creates a list of images
                def makeDatelist(d):
                    begin = ee.Date(d)
                    end = begin.advance(advance_overlap, timeframe)
                    imgcol = daily_mosaics.filterDate(begin, end)
                    monthsum= imgcol.reduce(ee.Reducer.max())\
                                    .set({'date':begin}) # Set date
                    return monthsum
                dummy = datelist.map(makeDatelist)

                # Turn list into image collection
                col = ee.ImageCollection(dummy)


                # Get rid of empty images
                c = col.map(lambda img: img.set('count', img.bandNames().length()))\
                       .filter(ee.Filter.gt('count', 0))

                # Function to get clear (1) and water (2)
                def getStats(image):
                    stats = image.select('water_max')\
                                 .unmask()\
                                 .add(image.select('clear_max').unmask())\
                                 .rename('stats')\
                                 .clip(bounds)
                    return image.addBands(stats).select('stats')

                # Apply function to get 'stats'
                new_stats = c.map(getStats)

                # Convert image collection to list
                im_list = new_stats.toList(new_stats.size())

                # Create list which is the length of the image collection
                imageNames = ee.List.sequence(0,im_list.length().subtract(1),1)
                plain_list = imageNames.getInfo()

                for i in plain_list:
                    img = ee.Image(im_list.get(i))
                    description = code_output+'_'+image+'_'+collection+'_'+index+'_gt'+str(threshold)+'_cplt_' +str(cloud_percent)\
                    +'_slc_'+str(slc_data)+'_clip57_'+str(clip_edges)+'_'+ee.Date(img.get('date')).format('YYYYMMdd').getInfo()
                    task = ee.batch.Export.image.toCloudStorage(image= img.clip(bounds),
                                                                description= description, 
                                                                bucket= bucket,
                                                                region= bounds,
                                                                crs=crs, 
                                                                crsTransform=crsTransform,
                                                                maxPixels= maxPixels,
                                                                skipEmptyTiles= True,
                                                                fileNamePrefix= 'Python_API/'+fileNamePrefix+description,
                                                                fileFormat= 'GeoTIFF')
                    if test_mode == 'yes':
                        pass
                    elif test_mode == 'no':
                        task.start()
                    print(f'Submitting {description}')
            time.sleep(30*60)
        except Exception as e:
            print('Problem with the export')
            print(e)  


Check progress in the [Task Manager](https://code.earthengine.google.com/tasks)

In [ ]:
# # Kill all tasks in emergency
# for task in ee.batch.Task.list():
#     task.cancel()

### Export water persistence summaries

In [ ]:
if code_output == 'water_persistence':
    
    start = str(start_date)+'-01-01'
    end = str(end_date+1)+'-01-01'

    # Do collection specific mapping
    l5c = l5.map(maskL457sr).map(renameBands57)
    if slc_data == 'yes':
        l7c = l7.map(maskL457sr).map(renameBands57)
    elif slc_data == 'no':
        l7c = l7.map(maskL457sr).map(renameBands57).filterDate('1999-01-01','2003-05-31') # Remove SLC error affected images
    l8c = l8.map(maskL457sr).map(renameBands89)
    l9c = l9.map(maskL457sr).map(renameBands89)
    if clip_edges == 'yes':
        col = (l5c.map(clipEdges)).merge(l7c.map(clipEdges)).merge(l8c).merge(l9c)\
                 .filterDate(start,end)\
                 .filterBounds(bounds)\
                 .filter(ee.Filter.lt('CLOUD_COVER',cloud_percent))\
                 .map(addMNDWI)\
                 .map(maskBad)
    elif clip_edges == 'no':
        col = l5c.merge(l7c).merge(l8c).merge(l9c)\
                 .filterDate(start,end)\
                 .filterBounds(bounds)\
                 .filter(ee.Filter.lt('CLOUD_COVER',cloud_percent))\
                 .map(addMNDWI)\
                 .map(maskBad)

    string_start = ee.Date(col.first().date()).format('YYYYMMdd').getInfo()
    string_end = ee.Date(col.sort('system:time_start',False).first().date()).format('YYYYMMdd').getInfo()

    ### Run this
    description = code_output+'_'+image+'_'+collection+'_'+index+'_gt'+str(threshold)+'_cplt_' +str(cloud_percent)\
                  +'_slc_'+str(slc_data)+'_clip57_'+str(clip_edges)+string_start+'_'+string_end

    ### Filter collection and apply functions - water
    water_mosaics = col.map(onlyW)\
                       .select('water')

    ### Filter collection and apply functions - count (using B3)
    count_mosaics = col.map(onlyClear)\
                       .select('clear')

    ###  Calculate percent of inundated (water/total observations)*100
    water = water_mosaics.count()
    count = count_mosaics.count()
    proportion = water.divide(count)
    percent= proportion.multiply(100)

    ###  Export water persistence
    ###  MNDWI gt 0 and gt 0.1
    ###  % gte 50, 60, 70, 80, 90 - wait no, they can just threshold it in arc
    ###  Export water persistence image
    task = ee.batch.Export.image.toCloudStorage(image=percent.clip(bounds),
                                        description= description, 
                                        bucket= bucket,
                                        region= bounds,
                                        crs=crs, 
                                        crsTransform=crsTransform,
                                        maxPixels= maxPixels,
                                        skipEmptyTiles= True,
                                        fileNamePrefix= 'Python_API/'+fileNamePrefix+description,
                                        fileFormat= 'GeoTIFF')
    if test_mode == 'yes':
        pass
    elif test_mode == 'no':
        task.start()
    print(f'Submitting {description}')
